In [1]:

# coding: utf-8

# In[ ]:


from __future__ import division


# In[ ]:


import tensorflow as tf
import numpy as np
import matplotlib.pyplot as pyplot
from sklearn.model_selection import train_test_split
import cv2
import os
from skimage import io as ski_io
from skimage.viewer import ImageViewer
from skimage import color as ski_color
from skimage import io as ski_io
import time
from nolearn.lasagne import BatchIterator
from skimage import transform as ski_transform
NUM_CLASS=10
def one_hot(labels, n_class = NUM_CLASS):
    total_record = len(labels)
    y = np.zeros((total_record, n_class))
    y[np.arange(total_record), labels] = 1
    return y


from collections import namedtuple

Parameters = namedtuple('Parameters', [
        # Data parameters
        'num_classes', 'image_size', 
        # Training parameters
        'batch_size', 'max_epochs', 'log_epoch', 'print_epoch',
        # Optimisations
        'learning_rate_decay', 'learning_rate',
        'l2_reg_enabled', 'l2_lambda', 
        'early_stopping_enabled', 'early_stopping_patience', 
        'resume_training', 
        # Layers architecture
        'conv1_k', 'conv1_d', 'conv1_p', 
        'conv2_k', 'conv2_d', 'conv2_p', 
        'conv3_k', 'conv3_d', 'conv3_p', 
        'fc4_size', 'fc4_p'
    ])

import os

class Paths(object):
    """
    Provides easy access to common paths we use for persisting 
    the data associated with model training.
    """
    
    def __init__(self, params):
        """
        Initialises a new `Paths` instance and creates corresponding folders if needed.

        Parameters
        ----------
        params  : Parameters
                  Structure (`namedtuple`) containing model parameters.
        """
        self.model_name = self.get_model_name(params)
        self.var_scope = self.get_variables_scope(params)
        self.root_path = os.getcwd() + "/models/" + self.model_name + "/"
        self.model_path = self.get_model_path()
        self.train_history_path = self.get_train_history_path()
        self.learning_curves_path = self.get_learning_curves_path()
        if not os.path.exists(self.root_path):
            os.makedirs(self.root_path)

    def get_model_name(self, params):
        """
        Generates a model name with some of the crucial model parameters encoded into the name.

        Parameters
        ----------
        params  : Parameters
                  Structure (`namedtuple`) containing model parameters.
                  
        Returns
        -------
        Model name.
        """
        # We will encode model settings in its name: architecture, optimisations applied, etc.
        print ("call get model name")
        model_name = "k{}d{}p{}_k{}d{}p{}_k{}d{}p{}_fc{}p{}".format(
            params.conv1_k, params.conv1_d, params.conv1_p, 
            params.conv2_k, params.conv2_d, params.conv2_p, 
            params.conv3_k, params.conv3_d, params.conv3_p, 
            params.fc4_size, params.fc4_p
        )
        model_name += "_lrdec" if params.learning_rate_decay else "_no-lrdec"
        model_name += "_l2" if params.l2_reg_enabled else "_no-l2"
        return model_name

    def get_variables_scope(self, params):
        """
        Generates a model variable scope with some of the crucial model parameters encoded.

        Parameters
        ----------
        params  : Parameters
                  Structure (`namedtuple`) containing model parameters.
                  
        Returns
        -------
        Variables scope name.
        """
        # We will encode model settings in its name: architecture, optimisations applied, etc.
        var_scope = "k{}d{}_k{}d{}_k{}d{}_fc{}_fc0".format(
            params.conv1_k, params.conv1_d,
            params.conv2_k, params.conv2_d,
            params.conv3_k, params.conv3_d, 
            params.fc4_size
        )
        return var_scope

    def get_model_path(self):
        """
        Generates path to the model file.
   
        Returns
        -------
        Model file path.
        """
        return self.root_path + "model.ckpt"

    def get_train_history_path(self):
        """
        Generates path to the train history file.
   
        Returns
        -------
        Train history file path.
        """
        return self.root_path + "train_history"
    
    def get_learning_curves_path(self):
        """
        Generates path to the learning curves graph file.
   
        Returns
        -------
        Learning curves file path.
        """
        return self.root_path + "learning_curves.png"


# In[ ]:


def fully_connected(input, size):
    """
    Performs a single fully connected layer pass, e.g. returns `input * weights + bias`.
    """
    weights = tf.get_variable( 'weights', 
        shape = [input.get_shape()[1], size],
        initializer = tf.contrib.layers.xavier_initializer()
      )
    biases = tf.get_variable( 'biases',
        shape = [size],
        initializer = tf.constant_initializer(0.0)
      )
    return tf.matmul(input, weights) + biases

def fully_connected_relu(input, size):
    return tf.nn.relu(fully_connected(input, size))

def conv_relu(input, kernel_size, depth):
    """
    Performs a single convolution layer pass.
    """
    weights = tf.get_variable( 'weights', 
        shape = [kernel_size, kernel_size, input.get_shape()[3], depth],
        initializer = tf.contrib.layers.xavier_initializer()
      )
    biases = tf.get_variable( 'biases',
        shape = [depth],
        initializer = tf.constant_initializer(0.0)
      )
    conv = tf.nn.conv2d(input, weights,
        strides = [1, 1, 1, 1], padding = 'SAME')
    return tf.nn.relu(conv + biases)

def pool(input, size):
    """
    Performs a max pooling layer pass.
    """
    return tf.nn.max_pool(
        input, 
        ksize = [1, size, size, 1], 
        strides = [1, size, size, 1], 
        padding = 'SAME'
    )

def model_pass(input, params, is_training):
    # Convolutions

    with tf.variable_scope('conv1'):
        conv1 = conv_relu(input, kernel_size = params.conv1_k, depth = params.conv1_d) 
    with tf.variable_scope('pool1'): 
        pool1 = pool(conv1, size = 2)
        pool1 = tf.cond(is_training, lambda: tf.nn.dropout(pool1, keep_prob = params.conv1_p), lambda: pool1)
    with tf.variable_scope('conv2'):
        conv2 = conv_relu(pool1, kernel_size = params.conv2_k, depth = params.conv2_d)
    with tf.variable_scope('pool2'):
        pool2 = pool(conv2, size = 2)
        pool2 = tf.cond(is_training, lambda: tf.nn.dropout(pool2, keep_prob = params.conv2_p), lambda: pool2)
    with tf.variable_scope('conv3'):
        conv3 = conv_relu(pool2, kernel_size = params.conv3_k, depth = params.conv3_d)
    with tf.variable_scope('pool3'):
        pool3 = pool(conv3, size = 2)
        pool3 = tf.cond(is_training, lambda: tf.nn.dropout(pool3, keep_prob = params.conv3_p), lambda: pool3)
    
    # Fully connected
    
    # 1st stage output
    pool1 = pool(pool1, size = 4)
    shape = pool1.get_shape().as_list()
    pool1 = tf.reshape(pool1, [-1, shape[1] * shape[2] * shape[3]])
    
    # 2nd stage output
    pool2 = pool(pool2, size = 2)
    shape = pool2.get_shape().as_list()
    pool2 = tf.reshape(pool2, [-1, shape[1] * shape[2] * shape[3]])    
    
    # 3rd stage output
    shape = pool3.get_shape().as_list()
    pool3 = tf.reshape(pool3, [-1, shape[1] * shape[2] * shape[3]])
    
    flattened = tf.concat([pool1, pool2, pool3], 1)
    
    with tf.variable_scope('fc4'):
        fc4 = fully_connected_relu(flattened, size = params.fc4_size)
        fc4 = tf.cond(is_training, lambda: tf.nn.dropout(fc4, keep_prob = params.fc4_p), lambda: fc4)
    with tf.variable_scope('out'):
        logits = fully_connected(fc4, size = params.num_classes)
    return logits


parameters = Parameters(
    # Data parameters
    num_classes = NUM_CLASS,
    image_size = (32, 32),
    # Training parameters
    batch_size = 256,
    max_epochs = 100,
    log_epoch = 1,
    print_epoch = 1,
    # Optimisations
    learning_rate_decay = False,
    learning_rate = 0.0001,
    l2_reg_enabled = True,
    l2_lambda = 0.0001,
    early_stopping_enabled = True,
    early_stopping_patience = 100,
    resume_training = True,
    # Layers architecture
    conv1_k = 5, conv1_d = 32, conv1_p = 0.9,
    conv2_k = 5, conv2_d = 64, conv2_p = 0.8,
    conv3_k = 5, conv3_d = 128, conv3_p = 0.7,
    fc4_size = 1024, fc4_p = 0.5
)

/opt/python-virtualenv/ml-python3/lib/python3.5/site-packages/skimage/viewer/__init__.py:6: UserWarning: Viewer requires Qt
  warn('Viewer requires Qt')
/opt/python-virtualenv/ml-python3/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
import cv2
import numpy as np

In [3]:
def image_resize(image, width = None, height = None, inter = cv2.INTER_AREA):
    # initialize the dimensions of the image to be resized and
    # grab the image size
    dim = None
    (h, w) = image.shape[:2]

    # if both the width and height are None, then return the
    # original image
    if width is None and height is None:
        return image

    # check to see if the width is None
    if width is None:
        # calculate the ratio of the height and construct the
        # dimensions
        r = height / float(h)
        dim = (int(w * r), height)

    # otherwise, the height is None
    else:
        # calculate the ratio of the width and construct the
        # dimensions
        r = width / float(w)
        dim = (width, int(h * r))

    # resize the image
    resized = cv2.resize(image, dim, interpolation = inter)

    # return the resized image
    return resized
def rotate(image, angle, center = None, scale = 1.0):
    (h, w) = image.shape[:2]

    if center is None:
        center = (w / 2, h / 2)

    # Perform the rotation
    M = cv2.getRotationMatrix2D(center, angle, scale)
    rotated = cv2.warpAffine(image, M, (w, h))

    return rotated

In [4]:
map_class = ['stop', 'turn_left', 'turn_right', 'prob_left', 'prob_right', 'no_entry', 'speed_limit', 'min_speed', 'end_min', 'unknown']
map_answer = []

In [5]:
# threshold
top_thres=40
bottom_thres=50
prev_check_thres=5
prev_w_thres=5
prev_h_thres=5
roi_id=0
rand_appear_w_thres=20
rand_appear_h_thres=20


In [17]:
import cv2
import numpy as np
cap = cv2.VideoCapture('./input-sample/MVI_1063.avi')
num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frame_rois = np.zeros([num_frames, 5, 5]) #num frame, max roi per frame, properties
# preperty

In [7]:
def backtrack(frame_id, x, y, w, h, depth):
    if depth >= prev_check_thres:
        return True
    if frame_id < 0:
        return False
    for frame_p in frame_rois[frame_id-1]:
        None

In [18]:
# Initialisation routines: generate variable scope, create logger, note start time.
paths = Paths(parameters)

# Build the graph
graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed at run time with a training minibatch.
    tf_x = tf.placeholder(tf.float32, shape = (None, parameters.image_size[0], parameters.image_size[1], 1))
    is_training = tf.constant(False)
    with tf.variable_scope(paths.var_scope):
        predictions = tf.nn.softmax(model_pass(tf_x, parameters, is_training))
        top_k_predictions = tf.nn.top_k(predictions, 5)

with tf.Session(graph = graph) as session:
    session.run(tf.global_variables_initializer())
    tf.train.Saver().restore(session, paths.model_path)        
    while (1):

        _, frame = cap.read()

        blur = cv2.medianBlur(frame.copy(), 5)

        # Convert BGR to HSV
        hsv = cv2.cvtColor(blur, cv2.COLOR_BGR2HSV)

        # define range of blue color in HSV
        lower_blue = np.array([100, 70, 40])
        upper_blue = np.array([130, 255, 255])

        lower_red = np.array([0, 70, 50])
        upper_red = np.array([10, 255, 255]) 
        r_mask1 = cv2.inRange(hsv, lower_red, upper_red) 
        r_mask2 = cv2.inRange(hsv, np.array([100, 10, 170]), np.array([180, 230, 255])) 
        r_mask3 = cv2.inRange(hsv, np.array([157, 60, 11]), np.array([180, 255, 255])) 
        red_mask = r_mask1 | r_mask2 | r_mask3
        

        # Threshold the HSV image to get only blue colors
        blue_mask = cv2.inRange(hsv, lower_blue, upper_blue)
        mask = blue_mask + red_mask

        # Bitwise-AND mask and original image
        res = cv2.bitwise_and(frame, frame, mask=mask)
        mask = cv2.erode(mask.copy(), None, iterations=2)
        mask2 = cv2.dilate(mask.copy(), None, iterations=2)

        cnts = cv2.findContours(mask2.copy(), cv2.RETR_EXTERNAL,
                                cv2.CHAIN_APPROX_SIMPLE)[-2]
        framedraw = frame.copy()
#         framedraw = cv2.cvtColor(framedraw, cv2.COLOR_BGR2GRAY)
#         framedraw = cv2.Canny(framedraw, 100, 200)
        if len(cnts) > 0:
            for cnt in cnts:
                x, y, w, h = cv2.boundingRect(cnt)
#                 Vi tri cac bien            
#                 if w < 20 or h < 20 or y < 30 or h / w > 2 or w / h > 1.5:
#                     cv2.rectangle(framedraw, (x, y), (x + w, y + h), (0, 255, 0), 2)
#                     continue
                roi = frame[y:y+h, x:x+w]
                print (y, x, y +h, x + w)
                # extract image and feed to cnn
#                 roi = cv2.cvtColor(roi,cv2.COLOR_BGR2RGB)  
#                 roi = ski_transform.resize(roi, (32, 32))
#                 roi = ski_color.rgb2gray(roi)
#                 roi = roi.reshape(roi.shape + (1,))        
#                 tf_X = np.array([roi])
#                 cv2.rectangle(framedraw, (x, y), (x + w, y + h), (255,0,0), 2)
#                 [prediction] = session.run([top_k_predictions], feed_dict = {
#                         tf_x : tf_X
#                     }
#                 )
#                 prob_class =prediction[0][0][0] 
#                 class_ord = prediction[1][0][0]
#                 class_name = map_class[class_ord]
#                 cv2.putText(framedraw, class_name, (x+w, y + int(h / 2)), cv2.FONT_HERSHEY_SIMPLEX, 1,color=(0,0,255))
        cv2.imshow('frame', framedraw)

        k = cv2.waitKey(1000000) & 0xFF
        if k == 27:
            break

cv2.destroyAllWindows()

call get model name
INFO:tensorflow:Restoring parameters from /home/tuyenlv17/data/programming/projects/digital-race-2/GSS-CNN-classification/models/k5d32p0.9_k5d64p0.8_k5d128p0.7_fc1024p0.5_no-lrdec_l2/model.ckpt
305 88 311 96
156 150 161 156
138 16 327 269
110 139 124 153
97 144 118 162
71 152 77 157
64 0 72 5
54 0 61 3
48 495 59 504
45 385 55 391
44 330 56 355
40 415 47 424
36 401 49 412
31 0 36 5
26 597 43 610
25 281 33 290
22 553 44 565
20 247 27 258
20 234 26 241
17 46 35 64
16 0 32 17
13 122 18 129
11 17 30 42
7 609 25 626
0 504 51 559
0 351 4 357
0 340 3 347
0 122 31 216
0 67 32 122
0 21 6 49
0 0 13 18
181 262 188 268
155 151 161 160
138 15 327 269
108 139 126 153
96 143 118 164
73 153 78 158
63 0 71 5
49 0 60 4
47 494 59 509
45 386 51 391
44 330 56 355
42 479 48 486
41 416 48 423
36 402 49 412
34 390 39 396
25 598 44 610
25 281 33 291
21 554 44 568
21 235 26 240
19 247 28 260
16 46 34 63
13 202 24 214
7 612 25 628
0 492 54 560
0 350 6 357
0 342 3 348
0 122 29 201
0 68 31 135
0

245 137 256 148
153 151 159 161
134 19 330 273
107 140 123 154
94 141 114 164
67 153 77 161
62 0 72 10
47 0 59 7
39 337 51 359
30 405 45 423
26 388 50 399
23 562 35 570
20 284 28 293
17 606 34 617
16 128 22 136
16 48 30 66
14 245 24 260
14 0 29 21
10 203 20 214
10 74 27 127
0 480 53 563
0 159 24 203
0 138 25 159
0 66 13 124
0 43 5 48
0 22 21 42
0 0 11 17
153 151 159 163
132 18 326 273
106 139 119 153
95 148 115 164
81 112 86 118
67 153 77 161
61 0 71 10
47 0 58 7
43 536 49 542
37 334 50 360
32 306 38 311
29 405 45 426
28 389 49 399
27 0 33 3
22 563 35 571
22 23 27 28
20 284 28 293
15 606 34 617
13 239 24 263
13 48 30 67
13 0 27 22
9 75 26 133
0 624 3 638
0 481 53 568
0 134 25 219
0 63 11 126
0 24 25 53
0 0 10 17
152 151 158 162
132 17 324 273
105 140 120 153
93 144 117 164
66 154 76 161
60 0 70 10
46 0 57 7
38 340 50 359
37 389 48 395
33 481 52 493
32 306 37 311
27 406 44 427
27 394 35 400
24 564 35 571
19 285 27 294
15 607 34 618
13 240 25 263
13 48 29 68
11 0 26 20
0 630 5 640
0 493 

281 50 303 120
118 50 277 274
90 131 108 146
76 136 99 156
50 200 56 205
47 157 56 164
39 0 48 15
22 0 36 18
21 491 38 500
21 342 31 367
19 426 26 434
16 413 30 423
2 292 10 301
1 627 6 632
0 637 3 640
0 502 38 580
0 251 5 263
0 192 5 211
0 173 3 183
0 151 4 167
0 118 4 130
0 91 4 101
0 56 6 74
0 28 3 40
0 13 3 19
0 0 9 6
283 58 301 118
142 142 147 148
118 49 279 273
91 130 108 145
77 136 99 156
61 113 67 118
47 156 57 162
41 0 49 14
31 470 36 477
30 492 35 497
24 346 29 351
24 0 37 18
21 353 34 368
20 491 27 498
19 426 26 436
16 412 29 423
3 291 10 300
2 0 11 6
0 637 4 640
0 625 5 633
0 502 37 575
0 305 3 311
0 249 9 263
0 191 6 210
0 150 5 187
0 115 5 134
0 109 6 114
0 93 7 108
0 56 8 72
0 27 4 38
0 12 5 26
299 50 304 55
285 57 303 117
143 144 148 150
119 47 281 273
91 130 110 148
78 132 104 154
48 157 56 162
41 0 50 14
31 471 36 477
26 0 38 18
24 547 29 552
22 345 33 366
19 490 35 499
17 412 30 435
11 506 17 511
3 290 11 301
3 0 11 6
0 626 4 635
0 501 37 573
0 304 3 312
0 250 9 269


151 126 158 141
121 37 297 273
94 112 115 130
83 117 105 143
49 161 59 168
42 17 50 25
28 550 33 557
27 560 36 568
25 406 33 411
25 354 32 361
25 16 39 30
23 364 36 376
20 502 29 512
20 481 26 488
15 422 32 447
4 300 12 310
0 504 41 592
0 312 5 324
0 257 10 274
0 234 4 244
0 209 4 222
0 104 9 191
0 96 8 102
0 66 10 81
0 37 7 62
0 25 7 36
151 128 158 140
121 35 298 272
95 113 115 129
81 115 112 140
48 162 58 168
42 17 50 25
27 16 39 31
25 406 34 412
24 363 36 375
24 355 33 362
22 560 27 565
20 502 29 515
15 423 32 447
4 301 12 312
2 96 8 102
1 198 6 204
0 506 41 600
0 313 5 325
0 258 6 275
0 235 3 241
0 206 6 220
0 103 9 193
0 66 9 82
0 38 6 62
0 26 6 37
153 124 158 129
151 135 157 141
122 34 301 272
107 131 113 138
95 111 116 130
80 116 105 142
48 162 58 168
42 17 50 25
27 407 33 412
25 16 39 31
24 355 32 362
23 365 35 377
21 504 28 515
20 481 25 489
14 518 21 523
14 423 32 446
4 301 12 311
2 96 9 102
0 510 41 596
0 313 4 325
0 262 6 275
0 235 4 244
0 210 4 224
0 105 9 193
0 68 9 83
0 

231 27 295 108
159 97 164 102
158 109 165 114
117 112 252 272
92 82 116 102
76 85 112 116
49 114 57 121
47 270 52 275
42 163 52 169
39 19 45 26
24 488 32 496
21 567 32 576
20 19 34 30
17 414 26 419
17 362 26 382
11 515 20 522
8 432 25 449
0 513 33 606
0 307 6 318
0 266 6 275
0 140 4 146
0 127 5 132
0 115 5 126
0 70 6 91
0 21 3 27
234 32 294 107
117 110 253 271
92 80 117 101
77 83 112 113
45 114 63 121
43 163 52 169
38 20 45 31
24 488 32 496
21 568 31 573
21 22 34 31
17 414 25 419
17 357 26 382
11 515 21 522
8 432 25 449
0 512 33 607
0 307 6 319
0 266 6 275
0 127 5 132
0 115 7 125
0 74 6 92
0 58 3 66
0 20 3 26
235 30 295 106
159 105 167 113
158 92 166 104
116 108 255 270
77 78 113 113
48 269 53 274
48 113 65 121
42 162 52 169
39 19 46 32
27 567 32 572
24 488 32 496
22 22 35 30
18 361 28 382
17 414 22 419
15 578 25 583
13 515 20 522
8 432 25 449
0 513 33 607
0 308 6 318
0 138 5 148
0 115 7 126
0 74 7 92
0 59 3 64
0 21 3 27
237 27 298 105
160 101 168 111
117 108 255 270
95 77 118 98
78 80

...

87 385 93 393
85 102 94 231
76 447 82 456
71 611 78 629
60 636 67 640
60 560 75 577
59 592 65 617
58 453 67 468
55 12 68 29
54 35 69 82
53 91 68 195
46 382 51 389
45 468 81 501
41 194 49 206
35 485 43 494
28 465 43 482
27 512 43 522
21 109 120 461
20 0 41 9
2 7 7 12
0 435 13 447
0 373 16 414
0 345 8 358
0 273 15 298
0 167 11 198
87 388 92 394
76 449 81 457
71 612 79 629
60 634 66 639
60 562 74 579
58 593 66 617
58 454 65 469
54 35 69 82
54 12 68 29
53 92 68 194
47 384 52 390
45 477 78 504
40 194 49 207
31 486 43 496
31 0 41 10
28 467 43 483
23 516 42 525
21 102 121 463
21 0 30 4
1 7 6 12
0 434 13 447
0 375 16 414
0 347 8 355
0 275 16 299
0 168 11 198
87 388 93 398
76 470 82 475
76 454 81 459
71 613 79 632
60 635 66 640
59 594 66 617
59 566 77 581
58 455 70 470
55 9 68 28
53 90 68 195
53 33 70 83
46 476 79 506
46 381 52 390
44 276 50 285
41 197 48 207
32 488 43 498
28 469 43 485
23 517 42 528
21 103 121 465
18 0 41 10
0 436 13 446
0 374 16 416
0 347 7 356
0 275 17 299
0 168 11 198
0 6 5

118 112 125 124
117 130 123 148
88 410 93 415
87 42 93 48
78 82 130 409
75 400 84 413
41 496 76 544
40 279 45 284
39 201 46 210
23 511 37 530
14 3 79 504
12 587 33 598
2 0 9 3
0 383 10 421
0 274 14 299
0 175 6 202
118 269 123 274
118 132 125 149
118 110 126 128
117 190 123 196
88 41 93 46
75 80 130 414
74 403 81 414
70 517 75 523
55 637 62 640
55 599 65 608
42 495 75 545
41 386 47 392
40 2 74 452
39 200 45 208
22 512 36 530
14 439 78 504
7 592 31 600
0 382 10 420
0 274 12 295
0 175 6 202
120 150 126 155
119 253 124 258
119 131 125 149
119 110 127 125
117 267 122 272
86 85 97 133
85 400 93 412
75 403 80 410
59 601 64 607
41 492 76 546
41 384 47 392
40 201 48 210
38 586 44 591
21 513 35 530
13 0 130 504
6 594 30 602
0 381 11 420
0 273 14 294
0 174 7 199
169 557 174 562
120 137 127 154
120 107 127 126
118 266 123 272
118 251 123 257
87 86 97 129
86 130 96 239
84 399 93 411
74 402 83 414
58 604 65 609
54 583 62 590
44 493 79 547
40 0 132 452
40 199 47 209
37 589 42 594
21 513 36 530
13 438

208 401 213 407
129 204 134 209
129 156 134 161
127 284 133 295
127 264 133 282
127 232 132 238
127 179 135 191
126 82 136 141
118 149 123 154
93 24 99 30
90 136 100 150
89 32 96 41
86 46 100 132
74 617 84 640
60 625 70 640
60 0 68 5
59 539 89 593
43 429 49 439
43 0 155 463
42 311 48 316
41 278 46 283
37 194 44 209
22 532 41 554
20 454 88 535
0 383 11 422
0 276 12 300
0 174 5 201
209 402 214 407
129 204 134 209
128 286 133 293
127 264 134 274
127 178 135 193
126 81 136 139
118 148 123 153
90 136 100 150
89 34 95 40
86 47 100 132
75 621 85 640
61 627 71 640
59 541 89 595
44 387 51 396
43 430 49 440
43 0 156 464
41 278 46 283
36 195 44 206
22 533 41 555
20 455 88 540
0 445 7 450
0 383 12 423
0 276 13 300
0 242 8 253
0 176 5 202
210 403 215 409
130 155 135 162
128 275 134 296
128 262 134 269
128 233 133 238
127 178 136 189
127 80 136 137
102 37 124 390
91 135 100 148
89 24 101 81
87 96 100 133
74 622 83 640
60 629 69 640
60 0 68 4
58 542 89 597
43 0 156 464
42 428 49 439
42 349 52 366
40 

245 426 254 440
146 147 151 152
145 105 151 116
144 48 153 83
142 190 148 198
141 167 146 174
140 248 147 272
140 224 146 237
140 200 147 209
139 283 148 302
131 130 136 135
120 415 182 478
111 401 119 413
109 0 133 401
104 134 111 145
99 143 104 154
99 88 112 125
99 12 112 80
93 155 111 271
56 583 88 633
44 380 50 389
44 0 110 471
43 424 49 435
43 272 48 277
40 189 47 202
21 544 38 567
17 463 97 583
0 442 5 454
0 377 13 416
0 346 5 352
0 270 16 294
0 230 12 245
0 164 9 196
248 428 256 442
146 48 154 81
145 138 151 143
145 100 154 113
144 40 149 46
143 190 148 195
143 167 151 175
142 224 148 231
142 198 149 208
141 250 148 268
140 280 148 302
121 416 183 480
110 0 135 416
103 156 112 166
101 8 115 79
100 89 113 124
93 180 112 271
77 588 82 593
70 620 84 632
55 594 86 635
44 383 49 389
43 423 49 434
43 271 49 277
43 81 48 86
41 518 96 586
41 189 48 202
17 544 37 567
16 0 110 539
0 442 3 450
0 376 13 415
0 342 5 351
0 269 16 293
0 235 4 241
0 162 10 193
250 430 259 443
150 145 155 150
14

312 475 329 506
212 313 217 319
163 121 168 136
163 88 170 104
162 167 167 172
159 181 168 190
159 158 167 166
159 17 170 48
158 247 167 260
157 266 167 316
157 202 162 210
153 466 235 550
145 117 150 124
123 0 150 454
118 264 123 269
117 271 122 276
117 154 122 160
108 0 123 18
107 95 123 129
107 21 123 82
101 201 123 273
55 539 60 544
49 430 54 441
49 354 54 359
47 548 111 640
43 194 51 208
27 573 45 596
21 0 122 566
0 540 7 548
0 442 13 463
0 353 18 427
0 274 19 303
0 236 15 248
0 149 13 203
317 477 333 511
213 314 218 320
163 134 168 139
163 120 169 133
163 88 170 104
159 182 168 190
159 155 168 175
158 278 167 315
158 246 167 261
158 16 170 48
155 470 238 555
145 116 150 123
123 0 150 456
107 96 122 128
107 16 123 82
107 0 117 17
98 202 123 275
49 448 54 454
49 428 55 442
49 0 122 494
49 355 54 360
48 550 112 640
43 194 50 209
28 575 45 598
22 483 107 568
0 541 6 549
0 443 11 463
0 354 18 428
0 275 19 306
0 237 14 249
0 148 12 204
319 481 337 514
163 119 169 138
163 16 169 21
162 